In [1]:
%matplotlib inline
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import to_categorical
from keras.models import load_model
import h5py
from keras.models import model_from_json
from scipy.misc import imread
import pylab

Using TensorFlow backend.


In [2]:
# Download the MNIST dataset if not present already
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Set some Hyperparameters
BATCH_SIZE = 128
NUM_EPOCHS = 5

# Image dimensions of the MNIST dataset
ROWS, COLS = 28, 28
NUM_CLASS = 10

In [4]:
# Define the Convolutional layers that act as feature detectors
feature_layers = [
    
    Conv2D(32, 5, padding='valid', input_shape=(ROWS, COLS, 1)),
    Activation('relu'),
    
    Conv2D(16, 5),
    Activation('relu'),

    MaxPooling2D(2),
    Dropout(0.25),
    Flatten(),
]

# Define the fully connected layers
classification_layers = [
    
    Dense(128),
    Activation('relu'),
    
    Dropout(0.5),
    Dense(NUM_CLASS),
    Activation('softmax')
]

In [5]:
# Build the Model and compile the Model
model = Sequential(feature_layers + classification_layers)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
# Use tensorboard to monitor the progress 
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=True)

In [7]:
def pre_process_data(x_data, y_data):
    """ Reshape the input images to (# of images, rows, cols, channels)
        Convert Y to categorical type using one-hot encoding
        return the new X and Y
    """
    X = x_data.reshape((x_data.shape[0], ROWS, COLS, 1)).astype('float32')
    X /= 255
    Y = to_categorical(y_data, NUM_CLASS)
    
    return X, Y

In [8]:
# Pre-process the training and the validation dataset
X_train, Y_train = pre_process_data(x_train, y_train)
X_valid, Y_valid = pre_process_data(x_test, y_test)

In [9]:
# Train the model
history = model.fit(X_train, Y_train,
              batch_size=BATCH_SIZE,
              epochs=NUM_EPOCHS,
              verbose=1,
              validation_data=(X_valid, Y_valid),
             callbacks=[tensorboard])

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 158s 3ms/step - loss: 0.2889 - acc: 0.9115 - val_loss: 0.0535 - val_acc: 0.9845
Epoch 2/5
60000/60000 [==============================] - 157s 3ms/step - loss: 0.0953 - acc: 0.9722 - val_loss: 0.0412 - val_acc: 0.9875
Epoch 3/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0711 - acc: 0.9784 - val_loss: 0.0328 - val_acc: 0.9891
Epoch 4/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0580 - acc: 0.9823 - val_loss: 0.0262 - val_acc: 0.9909
Epoch 5/5
60000/60000 [==============================] - 156s 3ms/step - loss: 0.0485 - acc: 0.9853 - val_loss: 0.0267 - val_acc: 0.9909


In [10]:
# Store the structure of the model into a json file
model_json = model.to_json()
with open("mnist_model.json", "w") as json_file:
    json_file.write(model_json)
    
# Store the learned weights of the parameter
model.save_weights("model.h5")


In [20]:
print("Accuracy of the model on the validation set : {}% ".format(history.history['val_acc'][-1] * 100))

Accuracy of the model on the validation set : 99.09% 
